In [3]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 50)
import numpy as np
import cost_function as cf
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'data-445-wagner'
bucket = s3.Bucket(bucket_name)

# Defining the file to be read from s3 bucket
file_key = "turnover_val.csv"

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# Reading the csv file
validation = pd.read_csv(file_content_stream)

# Defining the file to be read from s3 bucket
file_key1 = "turnover_test.csv"

bucket_object1 = bucket.Object(file_key1)
file_object1 = bucket_object1.get()
file_content_stream1 = file_object1.get('Body')

# Reading the csv file
test = pd.read_csv(file_content_stream1)

# Defining the file to be read from s3 bucket
file_key2 = "turnover_train.csv"

bucket_object2 = bucket.Object(file_key2)
file_object2 = bucket_object2.get()
file_content_stream2 = file_object2.get('Body')

# Reading the csv file
train = pd.read_csv(file_content_stream2)
train.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales,salary,left
0,0.92,0.95,6,239,4,0,0,sales,medium,0
1,0.88,0.89,4,254,5,0,0,sales,low,1
2,0.66,0.93,5,253,5,0,0,product_mng,low,1
3,0.46,0.45,2,172,2,1,0,management,low,0
4,0.88,0.75,5,152,3,0,0,hr,high,0


In [12]:
#Changing to dummy variables
validation = pd.concat([validation.drop(columns = ['sales'], axis = 1), pd.get_dummies(validation['sales'])], axis = 1)
validation = pd.concat([validation, pd.get_dummies(validation['salary'])], axis = 1)
train = pd.concat([train.drop(columns = ['sales'], axis = 1), pd.get_dummies(train['sales'])], axis = 1)
train = pd.concat([train, pd.get_dummies(train['salary'])], axis = 1)
test = pd.concat([test.drop(columns = ['sales'], axis = 1), pd.get_dummies(test['sales'])], axis = 1)
test = pd.concat([test, pd.get_dummies(test['salary'])], axis = 1)

validation.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,salary,left,IT,RandD,accounting,hr,management,marketing,product_mng,sales,support,technical,high,low,medium
0,0.83,0.50,5,274,3,0,0,low,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,0.80,0.72,3,271,2,0,0,medium,0,0,0,0,0,0,0,1,0,0,0,0,0,1
2,0.21,0.39,2,118,4,0,0,medium,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3,0.85,0.87,5,246,5,1,0,medium,1,0,0,0,0,0,0,0,1,0,0,0,0,1
4,0.46,0.53,2,147,3,0,0,low,1,0,0,0,0,0,0,0,1,0,0,0,1,0


In [13]:
#creating interactions
validation['Interaction_1'] = np.where((validation['satisfaction_level'] <= 0.465) & (validation['number_project'] <= 2.5) & (validation['last_evaluation'] <= 0.575), 1, 0)
validation['Interaction_2'] = np.where((validation['satisfaction_level'] <= 0.465) & (validation['number_project'] >= 2.5) & (validation['satisfaction_level'] <= 0.115), 1, 0)
validation['Interaction_3'] = np.where((validation['satisfaction_level'] >= 0.465) & (validation['time_spend_company'] <= 4.5) & (validation['average_montly_hours'] <= 290.5), 1, 0)
train['Interaction_1'] = np.where((train['satisfaction_level'] <= 0.465) & (train['number_project'] <= 2.5) & (train['last_evaluation'] <= 0.575), 1, 0)
train['Interaction_2'] = np.where((train['satisfaction_level'] <= 0.465) & (train['number_project'] >= 2.5) & (train['satisfaction_level'] <= 0.115), 1, 0)
train['Interaction_3'] = np.where((train['satisfaction_level'] >= 0.465) & (train['time_spend_company'] <= 4.5) & (train['average_montly_hours'] <= 290.5), 1, 0)
test['Interaction_1'] = np.where((test['satisfaction_level'] <= 0.465) & (test['number_project'] <= 2.5) & (test['last_evaluation'] <= 0.575), 1, 0)
test['Interaction_2'] = np.where((test['satisfaction_level'] <= 0.465) & (test['number_project'] >= 2.5) & (test['satisfaction_level'] <= 0.115), 1, 0)
test['Interaction_3'] = np.where((test['satisfaction_level'] >= 0.465) & (test['time_spend_company'] <= 4.5) & (test['average_montly_hours'] <= 290.5), 1, 0)


In [14]:
#Defining the input and target variables
X_train = train[['Interaction_3', 'Interaction_1', 'satisfaction_level', 'time_spend_company', 'number_project']]
Y_train = train['left']

X_val = validation[['Interaction_3', 'Interaction_1', 'satisfaction_level', 'time_spend_company', 'number_project']]
Y_val = validation['left']

X_test = test[['Interaction_3', 'Interaction_1', 'satisfaction_level', 'time_spend_company', 'number_project']]
Y_test = test['left']

#Scaling data
scaler = MinMaxScaler(feature_range = (0, 1))
X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)
X_test = scaler.fit_transform(X_test)


In [15]:
#parameters
rf_param_grid = {'n_estimators': [100, 300, 500], 'min_samples_split': [10, 15], 'min_samples_leaf': [5, 7], 'max_depth' : [3, 5, 7]}

#defining the custom scorer
my_score_function = make_scorer(cf.cost_function, greater_is_better = True, needs_proba = True)

#Random forrest model with GridSearchCV with 3 folds and the customized cost function
rf_grid_search = GridSearchCV(RandomForestClassifier(), rf_param_grid, cv = 3, scoring = my_score_function, n_jobs = -1).fit(X_train, Y_train)

#extracting best hyper parameters
rf_md = rf_grid_search.best_estimator_

In [16]:
#finding the optimal model
rf_val_pred = rf_md.predict_proba(X_val)[:, 1]
rf_test_pred = rf_md.predict_proba(X_test)[:, 1]

# making optimal cutoff
opt_cutoff = cf.cost_function_cutoff(Y_val, rf_val_pred)

#chaning to labels
rf_labels = np.where(rf_test_pred < opt_cutoff, 0, 1)

#Compute the cost of this prediction on the test data-frame
X = confusion_matrix(Y_test, rf_labels)
print(X)
print('\nThe cost of the RF model is:', -1500*X[1,0] - 1000*X[0, 1] + 500*X[1,1])

[[1130   13]
 [  28  329]]

The cost of the RF model is: 109500


In [17]:
#defining the parameter for svm
SVM_param_grid = {'kernel': ['rbf', 'poly', 'sigmoid'], 'C': [0.01, 0.1, 1, 10], 'gamma': [0.01, 0.1, 1]}

#SVM model with GridSearchCV with 3 folds and the customized cost function
svm_grid_search = GridSearchCV(SVC(probability = True), SVM_param_grid, cv = 3, scoring = my_score_function, n_jobs = -1).fit(X_train, Y_train)

#Extracting the best hyper-parameters
svm_md = svm_grid_search.best_estimator_

In [18]:
#Best SVM model
svm_val_pred = svm_md.predict_proba(X_val)[:, 1]
svm_test_pred = svm_md.predict_proba(X_test)[:, 1]

## Finding the optimal cutoff value by comparing the likelihoods of left in validation and the actual left values in 
## the validation
opt_cutoff = cf.cost_function_cutoff(Y_val, svm_val_pred)

## Use this cutoff to change the likelihoods of left in the test data-frame to label
svm_labels = np.where(svm_test_pred < opt_cutoff, 0, 1)

## Compute the cost of this prediction on the test data-frame
X = confusion_matrix(Y_test, svm_labels)
print(X)
print('\nThe cost of the SVC model is:', -1500*X[1,0] - 1000*X[0, 1] + 500*X[1,1])

[[1121   22]
 [  31  326]]

The cost of the SVC model is: 94500


In [ ]:
#The random forrest model is the one I would use as it has the better cost